In [6]:
import cv2
import tensorflow as tf
import numpy as np
import math
import time

In [7]:
model = tf.keras.models.load_model("mnist1.model")

In [8]:
def prepare(ima):
    IMG_SIZE = 28
    img_array=ima*255
    #img_array = cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    img_array=img_array/255.0  
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1,IMG_SIZE, IMG_SIZE,1)

def paa_skin(image,image1):  
    new1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dist = cv2.distanceTransform(new1, cv2.DIST_L2, 5)
    cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
    return image1,dist

def alpha(prediction):
    if(prediction==0):
        prediction='0'
    elif(prediction==1):
        prediction='1'
    elif(prediction==2):
        prediction='2'
    elif(prediction==3):
        prediction='3'
    elif(prediction==4):
        prediction='4'
    elif(prediction==5):
        prediction='5'
    elif(prediction==6):
        prediction='6'
    elif(prediction==7):
        prediction='7'
    elif(prediction==8):
        prediction='8'
    elif(prediction==9):
        prediction='9'
    else:
        prediction='nothing'
    return prediction

def callback(x):
    pass



In [11]:
cv2.namedWindow('trackbar')
cap=cv2.VideoCapture(0)
ilowH = 0
ihighH = 255
ilowS = 0
ihighS = 255
ilowV = 0
ihighV = 255
cv2.createTrackbar('lowH1', 'image1', ilowH, 179, callback)
cv2.createTrackbar('highH1', 'image1', ihighH, 179, callback)
cv2.createTrackbar('lowS1', 'image1', ilowS, 255, callback)
cv2.createTrackbar('highS1', 'image1', ihighS, 255, callback)
cv2.createTrackbar('lowV1', 'image1', ilowV, 255, callback)
cv2.createTrackbar('highV1', 'image1', ihighV, 255, callback)
img=[]
img_black=np.zeros((400,400))
img.append(img_black)
c=0
start=time.time()
while True:
    try:
        ret,frame=cap.read()
        frame=cv2.resize(frame, (400, 400), interpolation = cv2.INTER_LINEAR)
        
        ilowH1 = cv2.getTrackbarPos('lowH1', 'image1')
        ihighH1 = cv2.getTrackbarPos('highH1', 'image1')
        ilowS1 = cv2.getTrackbarPos('lowS1', 'image1')
        ihighS1 = cv2.getTrackbarPos('highS1', 'image1')
        ilowV1 = cv2.getTrackbarPos('lowV1', 'image1')
        ihighV1 = cv2.getTrackbarPos('highV1', 'image1')
        
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        low1=np.array([ilowH1,ilowS1,ilowV1])
        high1=np.array([ihighH1,ihighS1,ihighV1])
        img_mask1=cv2.inRange(hsv,low1,high1)
        output1=cv2.bitwise_and(frame,frame,mask=img_mask1)
        
        new1 = cv2.cvtColor(output1, cv2.COLOR_BGR2GRAY)
        dist = cv2.distanceTransform(new1, cv2.DIST_L2, 5)
        cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
        
        M=cv2.moments(dist)
        cX = (int(M["m10"] / M["m00"]))
        cY = (int(M["m01"] / M["m00"]))
        cv2.circle(frame, (cX, cY), 5, (0, 0, 255), -1)
        cv2.circle(img[c], (cX, cY), 5, (255, 255, 255),-1) 
        cv2.imshow("image1",frame)
        cv2.imshow('dist',dist)
        cv2.imshow("image",output1)
        end=time.time()
        diff=end-start
        cv2.imshow("black",img[c])
        
        if(diff>7):
            print("passing through the if condition")
            img[c]=255-img[c]
            img[c]=cv2.flip(img[c],1)
            cv2.imwrite("./array images/"+str(c)+".jpg",img[c])
            
            imge=cv2.imread("./array images/"+str(c)+".jpg");
        
            new1 = cv2.cvtColor(imge, cv2.COLOR_BGR2GRAY)
            prediction = model.predict([prepare(new1)])
            prediction=np.argmax(prediction)
            x=alpha(prediction)
            cv2.putText(imge,x,(60,60),cv2.FONT_HERSHEY_SIMPLEX,3.0,(0,255,255),lineType=cv2.LINE_AA)
            cv2.imwrite("digit.jpg",imge)
            imge=cv2.imread("digit.jpg")
            
            #cv2.waitKey(5000)            
            img_black=np.zeros((400,400))
            img.append(img1)
            c=c+1
            start=time.time()
        
        cv2.imshow("prediction",imge)    
    except:
        
        pass
    if (cv2.waitKey(1) == 13):
            break
cv2.destroyAllWindows()
cap.release()